In [158]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import poisson

import matplotlib.pyplot as plt

In [159]:
downweight_granularity=['dtime','Direction','dday','month']

In [160]:
Prediction_pax=pd.read_csv('R_Output_Training_Pax.csv',sep=',')
Prediction_pax=Prediction_pax[['Dprio']+downweight_granularity+['forecast_bookings']]\
.drop_duplicates(subset=['Dprio']+downweight_granularity)
Prediction_pax.index=range(Prediction_pax.shape[0])

Prediction_revenue=pd.read_csv('R_Output_Training_Revenue.csv',sep=',')
Prediction_revenue=Prediction_revenue.rename(columns={'forecast_bookings': 'forecast_revenue'})
Prediction_revenue=Prediction_revenue[['Dprio']+downweight_granularity+['forecast_revenue']]\
.drop_duplicates(subset=['Dprio']+downweight_granularity)
Prediction_revenue.index=range(Prediction_revenue.shape[0])

Prediction_pax=Prediction_pax.sort_values(by=downweight_granularity+['Dprio'])
Prediction_revenue=Prediction_revenue.sort_values(by=downweight_granularity+['Dprio'])

Prediction=pd.concat([Prediction_pax[[x for x in Prediction_pax.columns if x!='NumPax' and x!='forecast_node']],
                      Prediction_revenue['forecast_revenue']],axis=1)

Prediction['forecast_yield']=Prediction['forecast_revenue']/Prediction['forecast_bookings']

In [161]:
IdealCurve=pd.read_csv('IdealCurve_forYield.csv')

In [162]:
IdealCurve=IdealCurve.sort_values(by=downweight_granularity+['Dprio'])
Prediction=Prediction.sort_values(by=downweight_granularity+['Dprio'])

In [163]:
IdealCurve=pd.concat([IdealCurve,Prediction['forecast_yield']],axis=1)

In [164]:
IdealCurve['forecast_revenue_full']=IdealCurve['forecast_bookings_full']*IdealCurve['forecast_yield']
IdealCurve['forecast_revenue_downweighted']=IdealCurve['forecast_bookings_downweighted']*IdealCurve['forecast_yield']

In [165]:
IdealCurve=IdealCurve[downweight_granularity+['Dprio','forecast_revenue_full','forecast_revenue_downweighted']]

In [166]:
IdealCurve['Dprio']=-IdealCurve['Dprio']
IdealCurve=IdealCurve.set_index(['Dprio']+downweight_granularity)
IdealCurve_cumul_full=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_revenue_full'].cumsum().reset_index()
IdealCurve_cumul_full['Dprio']=-IdealCurve_cumul_full['Dprio']

In [167]:
IdealCurve=IdealCurve.reset_index()
IdealCurve=IdealCurve.set_index(['Dprio']+downweight_granularity)
IdealCurve_cumul_downweighted=IdealCurve.groupby(level=[0]+[x for x in range(1,len(downweight_granularity)+1)]).sum()\
.groupby(level=[x for x in range(1,len(downweight_granularity)+1)])['forecast_revenue_downweighted'].cumsum().reset_index()
IdealCurve_cumul_downweighted['Dprio']=-IdealCurve_cumul_downweighted['Dprio']

In [175]:
Data=pd.read_csv('R_Output_Test_Revenue.csv',sep=',')

In [176]:
Data=Data[['DepDate','yday','dtime','Direction','month','dday','Dprio','Coupons.YQ.on.current.bookings']]

Map_Dates=DataFrame([Data['DepDate'].unique().tolist(),[pd.to_datetime(x) for x in Data['DepDate'].unique().tolist()]]).transpose()
Map_Dates.columns=['DepDate','DepDate_dt']
Map_Dates['DepDate']=Map_Dates['DepDate'].astype('str')

Data=Data.merge(Map_Dates,on=['DepDate'])
Data['DepDate']=Data['DepDate_dt']
Data=Data[[x for x in Data.columns if x!='DepDate_dt']]

Data['TicketDate']=Data['DepDate']-Data['Dprio'].apply(lambda x: datetime.timedelta(x))

Data=Data.loc[Data['TicketDate']<=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]

Data=Data[[x for x in Data.columns if x!='TicketDate']]

Data['Dprio']=-Data['Dprio']
Data=Data.set_index(['DepDate','yday','dtime','Direction','month','dday','Dprio'])
Data=Data.groupby(level=[0,1,2,3,4,5,6]).sum().groupby(level=[0,1,2,3,4,5])['Coupons.YQ.on.current.bookings'].cumsum().reset_index()

Data['Dprio']=-Data['Dprio']

In [181]:
Data=Data.rename(columns={'Coupons.YQ.on.current.bookings': 'ActualRevenue'})

In [184]:
Data=Data.merge(IdealCurve_cumul_full,on=downweight_granularity+['Dprio'])
Data=Data.merge(IdealCurve_cumul_downweighted,on=downweight_granularity+['Dprio'])

In [194]:
Data.loc[Data['DepDate']=='2019-04-27',:]

,DepDate,yday,dtime,Direction,month,dday,Dprio,ActualRevenue,forecast_revenue_full,forecast_revenue_downweighted
12902,2019-04-27,117,405,O,4,Saturday,365,0,0.070510,0.048349
12905,2019-04-27,117,405,O,4,Saturday,364,0,0.142496,0.097712
12908,2019-04-27,117,405,O,4,Saturday,363,0,0.215991,0.148108
12911,2019-04-27,117,405,O,4,Saturday,362,0,0.291024,0.199560
12914,2019-04-27,117,405,O,4,Saturday,361,0,0.367630,0.252089
12917,2019-04-27,117,405,O,4,Saturday,360,0,0.445840,0.305719
12920,2019-04-27,117,405,O,4,Saturday,359,0,0.525688,0.360472
12923,2019-04-27,117,405,O,4,Saturday,358,0,0.607209,0.416372
12926,2019-04-27,117,405,O,4,Saturday,357,0,0.690437,0.473443
12929,2019-04-27,117,405,O,4,Saturday,356,0,0.775409,0.531709
